In [2]:
import pandas as pd
import numpy as np
import sys
import os
import string
import nltk
import time
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split


In [3]:
data = pd.read_csv('spam.csv',encoding='latin-1')
data.describe()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [4]:
#no empty cell value for column v1 and v2 , therefore proceed further
def processed(mail):
    temp =[]
    stem = PorterStemmer()
    words = word_tokenize(mail)
    for word in words:
        if word not in stop_words and word not in punctuations:
            temp.append(stem.stem(word.lower()))
    return ' '.join(temp)

In [5]:
# email = data["v2"]

In [6]:
#preprocessing the data
punctuations = string.punctuation
stop_words = list(stopwords.words("english"))
data["v2"] = data["v2"].apply(lambda x: processed(x))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Rishi/nltk_data'
    - 'c:\\Users\\Rishi\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\Rishi\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\Rishi\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\Rishi\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [7]:
# encoding the target labels in 0 and 1 
data["v1"] = data["v1"].replace('ham',0)
data["v1"] = data["v1"].replace('spam',1)

C:\Users\Rishi\AppData\Local\Temp\ipykernel_11676\3610196283.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["v1"] = data["v1"].replace('spam',1)


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# importing libraries for feature selection---------
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

In [9]:
tf = TfidfVectorizer()
tf_idf = tf.fit_transform(data["v2"])
x_train,x_test,y_train,y_test = train_test_split(tf_idf,data["v1"],test_size =0.3,random_state =1)

In [10]:
%time classifier_logistic = LogisticRegression(solver ='liblinear').fit(x_train,y_train)
classifier_NB = MultinomialNB().fit(x_train,y_train)
classifier_svm = SVC(kernel = 'linear')
classifier_svm.fit(x_train,y_train)
# rfecv = RFECV(estimator =classifier_svm,step = 1,cv = StratifiedKFold(2), scoring ='accuracy')
# rfecv.fit(x_train,y_train)

CPU times: total: 0 ns
Wall time: 25 ms


SVC(kernel='linear')

In [11]:
predicted_value_logistic = classifier_logistic.predict(x_test)
predicted_value_NB = classifier_NB.predict(x_test)
predicted_value_svm = classifier_svm.predict(x_test)

In [12]:
from sklearn import metrics
print(metrics.accuracy_score(y_test,predicted_value_logistic))
print(metrics.accuracy_score(y_test,predicted_value_NB))
print(metrics.accuracy_score(y_test,predicted_value_svm))

0.9611244019138756
0.9569377990430622
0.9808612440191388


In [13]:
# we see that SVM performs better than the other two classsifiers in terms of accuracy